# Question 1

We want to estimate the parameter $\lambda$  of an Exponential distribution using the generalized
method of moments. The probability density function of the Exponential distribution is given
by

$$f(x) =  \lambda_0 \cdot exp(-\lambda_0 \cdot x)$$ 
In our GMM, we will use the first and second moments of the exponential distribution:
$$\mathbb{E}[g(\lambda_0,X)]=\begin{bmatrix}
\frac{1}{\lambda_0}-X \\
\frac{1}{\lambda_0^{2}}-(X-\frac{1}{\lambda_0})^2
\end{bmatrix}=0
$$

1. Derive the gradient $$G(\lambda) = \frac{\partial{\mathbb{E}}[g ( \lambda,X)]}{\partial\lambda}$$
2. Setting your seed to 220122, randomly draw 100,000 observations from an exponential distribution with  $\lambda_0 = 5$. Using this vector as your data, estimate $\lambda_0$ using a GMM estimator where you provide the correctly specified gradient function.1 Report your estimated coeficient and its estimated standard error.


**1.1. Derive the gradient $$G(\lambda) = \frac{\partial{\mathbb{E}}[g ( \lambda,X)]}{\partial\lambda}$$**

$$\mathbb{E}[g(\lambda,X)]=\mathbb{E}\begin{bmatrix}
\frac{1}{\lambda}-X \\
\frac{1}{\lambda^{2}}-(X-\frac{1}{\lambda})^2
\end{bmatrix}=\begin{bmatrix}
\frac{1}{\lambda}-\mathbb{E}[X]\\
\frac{1}{\lambda^{2}}-\frac{1}{\lambda^{2}}+\mathbb{E}[\frac{2}{\lambda}X]-\mathbb{E}[X^2]
\end{bmatrix}=\begin{bmatrix}
\frac{1}{\lambda}-\frac{1}{\lambda_0}\\
\frac{2}{\lambda \cdot\lambda_0}-\frac{2}{\lambda_0^{2}}
\end{bmatrix}
$$

$$G(\lambda)=\begin{bmatrix}
\frac{\partial}{\partial \lambda}\left(\frac{1}{\lambda}-\frac{1}{\lambda_0}\right) \\
\frac{\partial}{\partial \lambda}\left(\frac{2}{\lambda \cdot\lambda_0}-\frac{2}{\lambda_0^{2}}\right)
\end{bmatrix}=\begin{bmatrix}
\frac{-1}{\lambda^{2}}\\
\frac{-2}{\lambda_0 \lambda^{2}}
\end{bmatrix}$$

**1.2. Setting your seed to 220122, randomly draw 100,000 observations from an exponential distribution with  $\lambda_0 = 5$. Using this vector as your data, estimate $\lambda_0$ using a GMM estimator where you provide the correctly specified gradient function.1 Report your estimated coeficient and its estimated standard error.**


In [1]:
#Cleaning working environment
rm(list=ls())
#Setting working directory
setwd("/Users/pedrogiraodesouza/Documents/Mestrado EESP - 2023/Problem Set 5/data")

#Loading Packages

library("AER")
library("dynlm") 
library("forecast") 
library("readxl")
library("stargazer")  
library("scales")
library("quantmod")
library("urca")
library(readr)
library("vars")
library('ggplot2')
library("gmm")
library("zoo")

#Setting Seed to 220122
set.seed(220122)

Carregando pacotes exigidos: car

Carregando pacotes exigidos: carData

Carregando pacotes exigidos: lmtest

Carregando pacotes exigidos: zoo


Attaching package: ‘zoo’


The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric


Carregando pacotes exigidos: sandwich

Carregando pacotes exigidos: survival

Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 


Please cite as: 


 Hlavac, Marek (2022). stargazer: Well-Formatted Regression and Summary Statistics Tables.

 R package version 5.2.3. https://CRAN.R-project.org/package=stargazer 


Carregando pacotes exigidos: xts

Carregando pacotes exigidos: TTR


Attaching package: ‘readr’


The following object is masked from ‘package:scales’:

    col_factor


Carregando pacotes exigidos: MASS

Carregando pacotes exigidos: strucchange



In [2]:
#Generating sample
mc_sample1<- rexp( 100000, 5)

#Defining Moment Conditions
g <- function( lambda, x ){
    m1 <- (1 / lambda) - x
    m2 <- (1/( lambda^2)) - (1/lambda - x)^2
    out <- cbind(m1, m2)
    return(out)
    
}
#Defining the "Gradient" vector
J <- function(lambda, x){
    j1<- -1/(lambda^2)
    j2<- -2/(mean(x) * lambda^2)
    grad <- as.matrix(c(j1,j2))
    return(grad)
}


In [3]:
model1 <- gmm(g,mc_sample1,t0=10, grad=J, method="Brent", lower=1, upper=20)

summary(model1)$coefficients

,Estimate,Std. Error,t value,Pr(>|t|)
Theta[1],4.970329,0.0006381623,7788.503,0


## Question 2
Let $\{Y_t\}$ be a stationary stochastic process such that
$$Y_t =  \phi_1\cdot Y_{t-1}+U_t$$
and
$$Ut =  \epsilon_t + \theta_1\cdot \epsilon_{t-2}+\theta_2\cdot \epsilon_{t-2}$$
where $\phi_1$;$\theta_1$;$\theta_2$ > 0 and  $\epsilon_t \sim$ i.i.d. N (0, 1).
1. Show that you cannot estimate Equation 1 using OLS. (
2. Show that $Y_{t-3}$ is not a valid instrument for $Y_{t-1}$ in Equation 1.
3. Show that $Y_{t-3}$ is a valid instrument for $Y_{t-1}$ in Equation 1.
4. Setting your seed to 220122, randomly draw 100,000 observations from ARMA(1,2) process where  $\phi_1 = 0.2$,  $\theta_1 = 0.1$,  $\theta_2 = 0.1$. Using this vector as your data, estimate $\phi_1$ and the intercept using a GMM estimator where $Y_{t-3}$ is used as an instrument for $Y_{t-1}$. Report the estimated coefficients (intercept and $\phi_1$) and their estimated standard errors.

**2.1 Show that you cannot estimate Equation 1 using OLS.**

For the standard OLS regression without intercept, the slope coefficient is given by
$$ \hat{\beta} =\frac{\sum x_i\cdot y_i}{\sum y_i^{2}}$$
where 
 - $\hat{\beta}$ is the slope coefficient estimate
 - $y_i$ is  realized value of the regressand on observation $i$
 - $x_i$ is the realized value of the regressor on observatio $i$
 
For an OLS estimation of the $\{Y_t\}$ stochastic process, we would have
$$ \hat{\phi}= \frac{\sum y_{t-1}\cdot y_t}{\sum y_{t-1}^{2}}=\frac{\sum y_{t-1}\cdot (\phi_1 y_{t-1}+u_t)}{\sum y_{t-1}^{2}}=\phi_1+\frac{\sum y_{t-1}\cdot u_t}{\sum y_{t-1}^{2}}$$
since $cov(Y_{t-1},U_{t})\neq 0$ and $\lim_p \hat{\phi}\neq \phi_1$
such that the OLS estimate of $\phi_1$ is not consistent.

**2.2 Show that $Y_{t-2}$ is not a valid instrument for $Y_{t-1}$ in Equation 1.**
For $Y_{t-2}$ to be a valid instrument for $Y_{t-1}$ in the regression of $Y_t$ it would need to satisfy 2 restrictions:

- The relevance restriction: $cov(Y_{t-2},Y_{t-1})\neq 0$
- The exclusion restrition: $cov(Y_{t-2},U_{t})=0$

Since $cov(U_t, Y_{t-2})=\theta_2\cdot cov(\epsilon_{t-2},\epsilon_{t-2})\neq 0$, the exclusion restriction would be violated, such that $Y_{t-2}$ would not be a valid instrument.


**2.3 Show that $Y_{t-3}$ is a valid instrument for $Y_{t-1}$ in Equation 1.**

As mentioned before, for $Y_{t-3}$ to be a valid instrument for $Y_{t-1}$ in this case, it would need to satisfy the relevance and exclusion restrictions.

- The relevance restriction
$$cov(Y_{t-1},Y_{t-3})=cov(\phi_1 \cdot Y_{t-2}+U_{t-1},Y_{t-3})=\phi_1 cov(Y_{t-2},Y_{t-3})+cov(U_{t-3},U_{t-1})\neq 0 $$

- The exclusion restriction
$$ cov(U_{t},Y_{t-3})= cov(\epsilon_t +\theta_1 \epsilon_{t-1} +\theta_2 \epsilon_{t-2}, Y_{t-1})=0$$

Consequently, Y_{t-3} would be a valid instrument.

**2.4 Setting your seed to 220122, randomly draw 100,000 observations from ARMA(1,2) process where   𝜙1=0.2 ,   𝜃1=0.1 ,   𝜃2=0.1 . Using this vector as your data, estimate 𝜙1  and the intercept using a GMM estimator where  𝑌𝑡−3  is used as an instrument for  𝑌𝑡−1 . Report the estimated coefficients (intercept and  𝜙1 ) and their estimated standard errors**

As the standard 2SLS is a particular case of GMM estimators, we'll use it as our 

In [4]:
phi1=0.2
theta1=0.1
theta2=0.1
#Generating tbe simulated sample
mc_sample2 <- arima.sim(model = list(ar = 0.2, ma=c(0.1,0.1)) , n = 100000)

#First stage
m1<- dynlm(L(mc_sample2,1)~L(mc_sample2,3))
#Second stage
m2 <-dynlm(mc_sample2~fitted(m1))
summary(m2)


Time series regression with "ts" data:
Start = 4, End = 1e+05

Call:
dynlm(formula = mc_sample2 ~ fitted(m1))

Residuals:
    Min      1Q  Median      3Q     Max 
-4.3404 -0.7075  0.0009  0.7100  4.9251 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) -0.004583   0.003336  -1.374    0.169    
fitted(m1)   0.173457   0.021790   7.960 1.74e-15 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 1.054 on 99995 degrees of freedom
Multiple R-squared:  0.0006333,	Adjusted R-squared:  0.0006233 
F-statistic: 63.36 on 1 and 99995 DF,  p-value: 1.735e-15


## Question 3

1. Files *BVSP.csv*, *BBDC3.csv*, *ABEV3.csv* and *ITUB3* come from Yahoo Finance and contain information about the IBOVESPA index and the ordinary shares of Bradesco, AMBEV and Ita u. In those  les, you can  nd the opening price, the closing price, the dailyhighest price, the daily lowest price, the adjusted closing price and the trading volume. Create one data set with daily returns (based on closing prices) during 2021.
2. File *selic.csv* comes from the Brazilian Central Bank and contain information about the Selic Rate. In this  le, you can  nd information about the annual Selic rate, the daily Selic factor, the daily trade volume, the number of daily operations, the daily mean, the daily median, the daily mode, the daily standard deviation and the daily kurtosis. Compute the daily Selic rate and merge it with the dataset created in the previous item. You should keep only the data from 2021. 
3. Create the variable ($R_{m,t} - R_{f,t}$) and the variables ($R_{i,t}- R_{f,t}$) for each share $i$ in your dataset.
4. Estimate the unrestricted CAPM model using GMM and the dataset built in the last item. Report all 6 coefficients and their standard errors.
5. Test the validity of the CAPM model by testing the null hypothesis $H_0$ : $\alpha_1 = 0$, $\alpha_2 = 0$,$\alpha_3 = 0$. Report the p-value of this asymptotic test. 


**3.1. Files *BVSP.csv*, *BBDC3.csv*, *ABEV3.csv* and *ITUB3* come from Yahoo Finance and contain information about the IBOVESPA index and the ordinary shares of Bradesco, AMBEV and Ita u. In those  les, you can  nd the opening price, the closing price, the dailyhighest price, the daily lowest price, the adjusted closing price and the trading volume. Create one data set with daily returns (based on closing prices) during 2021.**

In [5]:
#Loading data
bvsp <- read_csv("BVSP.csv")
bbdc <- read_csv("BBDC3.csv")
abev<- read_csv("ABEV3.csv")
itub <- read_csv("ITUB3.csv")
bvsp

#Creating time series
bvsp_ts <- xts(bvsp$Close,order.by=bvsp$Date)
bbdc_ts <- xts(bbdc$Close,order.by=bbdc$Date)
abev_ts <- xts(abev$Close,order.by=abev$Date)
itub_ts <- xts(itub$Close,order.by=itub$Date)




Rows: 247 Columns: 7
── Column specification ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
dbl  (6): Open, High, Low, Close, Adj Close, Volume
date (1): Date

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 247 Columns: 7
── Column specification ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
dbl  (6): Open, High, Low, Close, Adj Close, Volume
date (1): Date

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 247 Columns: 7
── Column specification ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
dbl  (6): Open, 

Date,Open,High,Low,Close,Adj Close,Volume
<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2021-01-04,119024,120354,118062,118558,118558,8741400
2021-01-05,118835,119790,116756,119223,119223,9257100
2021-01-06,119377,120924,118917,119851,119851,11638200
2021-01-07,119103,121983,119101,121956,121956,11774800
2021-01-08,122387,125324,122386,125077,125077,11085800
2021-01-11,125075,125075,122506,122807,122807,9537600
2021-01-12,123255,124584,123227,123998,123998,8949000
2021-01-13,123996,124032,121016,122040,122040,10291500
2021-01-14,121947,123896,121947,123481,123481,8974400


In [6]:
#bvsp<- xts(bvsp[,-1],order.by=bvsp)

stock_return <- function(stock_ts){
    
    d_p<- diff(stock_ts)[-1]
    s_returns <- d_p/lag(stock_ts)
    return(s_returns)
  }  

r_bvsp<-stock_return(bvsp_ts)
r_bbdc<-stock_return(bbdc_ts)
r_abev <- stock_return(abev_ts)
r_itub <- stock_return(itub_ts)

**3.2. File *selic.csv* comes from the Brazilian Central Bank and contain information about the Selic Rate. In this  le, you can  nd information about the annual Selic rate, the daily Selic factor, the daily trade volume, the number of daily operations, the daily mean, the daily median, the daily mode, the daily standard deviation and the daily kurtosis. Compute the daily Selic rate and merge it with the dataset created in the previous item. You should keep only the data from 2021.**

In [7]:
selic <- read.csv("selic.csv", sep=';')

#Cleaning data
selic$dfactor<-selic$Fator.diário

selic$dfactor <-as.numeric( gsub(",",".",selic$dfactor))
selic$Data <- as.Date(selic$Data, format="%d/%m/%Y")


#Creating time series 
dfactor_xts <- xts(selic$dfactor, order.by=selic$Data)

r_selic <- (dfactor_xts-1)["2021"][-1]


**3.3. Create the variable ($R_{m,t} - R_{f,t}$) and the variables ($R_{i,t}- R_{f,t}$) for each share $i$ in your dataset.**


In [8]:
er_m=r_bvsp-r_selic
er_bbdc= r_bbdc-r_selic
er_abev= r_abev-r_selic
er_itub= r_itub - r_selic

**3.4. Estimate the unrestricted CAPM model using GMM and the dataset built in the last item. Report all 6 coefficients and their standard errors.**

In [9]:
g2<-function(beta,x){
    m11 <- (x[,1]-beta[1]-beta[2]*er_m)
    m21 <- (x[,1]-beta[1]-beta[2]*er_m)*er_m
    m12 <- (x[,2]-beta[3]-beta[4]*er_m)
    m22 <- (x[,2]-beta[3]-beta[4]*er_m)*er_m
    m13 <- (x[,3]-beta[5]-beta[6]*er_m)
    m23 <- (x[,2]-beta[5]-beta[6]*er_m)*er_m
    return(cbind(m11,m21,m12,m22,m13,m23))

}

J2<-function(beta,x){
    a11<-1
    a12<- mean(er_m)
    b11<-mean(er_m)
    b12<- mean(er_m^2)
    a21<-1
    a22<- mean(er_m)
    b21<-mean(er_m)
    b22<- mean(er_m^2)
    a31<-1
    a32<- mean(er_m)
    b31<-mean(er_m)
    b32<- mean(er_m^2)
    jacob<-matrix(c(a11,b11,0,0,0,0,
                    a12,b12,0,0,0,0,
                    0,0,a21,b22,0,0,
                    0,0,a22,b22,0,0,
                    0,0,0,0,a31,b31,
                    0,0,0,0,a32,b32
                    ),nrow=6,ncol=6)
    return(jacob)
    
}
x<-cbind(er_bbdc,er_abev,er_itub)
ucapm<-gmm(g2,x=x,grad=J2,t0=rep(1,6))
summary(ucapm)


Call:
gmm(g = g2, x = x, t0 = rep(1, 6), gradv = J2)


Method:  twoStep 

Kernel:  Quadratic Spectral

Coefficients:
          Estimate     Std. Error   t value      Pr(>|t|)   
Theta[1]  -9.0891e-05   9.7779e-04  -9.2956e-02   9.2594e-01
Theta[2]   1.9642e+00   1.0557e-01   1.8606e+01   2.8648e-77
Theta[3]   1.0194e-03   1.2074e-03   8.4429e-01   3.9851e-01
Theta[4]   1.7435e+00   1.4800e-01   1.1780e+01   4.9453e-32
Theta[5]  -6.1991e-04   1.0331e-03  -6.0004e-01   5.4848e-01
Theta[6]   1.3666e+00   1.1491e-01   1.1894e+01   1.2786e-32

J-Test: degrees of freedom is 0 
                J-test                P-value             
Test E(g)=0:    1.72419707690528e-05  *******             

#############
Information related to the numerical optimization
Convergence code =  0 
Function eval. =  195 
Gradian eval. =  NA 

**3.5. Test the validity of the CAPM model by testing the null hypothesis $H_0$ : $\alpha_1 = 0$, $\alpha_2 = 0$,$\alpha_3 = 0$. Report the p-value of this asymptotic test.**

In [10]:
capm<-gmmWithConst(ucapm,c(1,3,5),c(0,0,0 ))
summary(capm)


Call:
gmmWithConst(obj = ucapm, which = c(1, 3, 5), value = c(0, 0, 
    0))


Method:  twoStep (with equality constraints) 

Kernel:  Quadratic Spectral(with bw =  0.85244 )

Coefficients:
          Estimate    Std. Error  t value     Pr(>|t|)  
Theta[2]  1.1154e+00  5.8807e-02  1.8967e+01  3.2025e-80
Theta[4]  8.2796e-01  7.4929e-02  1.1050e+01  2.1958e-28
Theta[6]  7.8700e-01  7.3962e-02  1.0640e+01  1.9314e-26

J-Test: degrees of freedom is 3 
                J-test   P-value
Test E(g)=0:    0.59817  0.89685

Initial values of the coefficients
  Theta[2]   Theta[4]   Theta[6] 
1.86095194 0.07943224 2.27472068 

#### Equality constraints ####
Theta[1]  =  0 
Theta[3]  =  0 
Theta[5]  =  0 
##############################

#############
Information related to the numerical optimization
Convergence code =  0 
Function eval. =  124 
Gradian eval. =  NA 